[Reference](https://towardsdev.com/farm-stack-the-best-development-suite-for-python-full-stack-development-1a8d74660e4)

```
fastapi==0.95.2
uvicorn==0.22.0
pymongo==4.3.3
```

In [1]:
from pymongo import MongoClient

db_connection = MongoClient("mongodb://localhost:27017")
db = db_connection["nobel_prize_db"]
collection = db["nobel_prize"]

In [2]:
from pydantic import BaseModel

from models.LaureatesModel import Laureates


class NobelPrize(BaseModel):
    year: str
    category: str
    laureates: list[Laureates]

In [3]:
from pydantic import BaseModel


class Laureates(BaseModel):
    firstname: str
    surname: str
    motivation: str
    share: str

In [4]:
from models.LaureatesModel import Laureates
from models.NobelPrizeModel import NobelPrize
import random


def nobel_prize_serializer(nobel_prize) -> dict:
    try:
        return {
            'id': str(nobel_prize['_id']),
            'year': str(nobel_prize['year']),
            'category': str(nobel_prize['category']),
            'laureates': list(nobel_prize_laureates_serializer(nobel_prize['laureates']))
        }
    except Exception as e:
        print(e.__cause__)


def nobel_prize_laureate_serializer(laureate) -> dict:
    try:
        return {
            'id': str(laureate['id']),
            'firstname': str(laureate['firstname']),
            'surname': str(laureate['surname']),
            'motivation': str(laureate['motivation']),
            'share': str(laureate['share'])
        }
    except Exception as e:
        print(e.__cause__)


def nobel_prize_laureates_serializer(laureates) -> list:
    try:
        return [nobel_prize_laureate_serializer(laureate) for laureate in laureates]
    except Exception as e:
        print(e.__cause__)


def nobel_prizes_serializer(nobel_prizes) -> list:
    try:
        return [nobel_prize_serializer(nobel_prize) for nobel_prize in nobel_prizes]
    except Exception as e:
        print(e.__cause__)


def serialize_payload(payload: NobelPrize) -> dict:
    try:
        return {
            'year': str(payload.year),
            'category': str(payload.category),
            'laureates': serialize_payload_laureates_arrays(payload.laureates)
        }
    except Exception as e:
        print(e.__cause__)


def serialize_payload_laureates_arrays(laureates) -> list:
    return [serialize_payload_laureates_object(laureate) for laureate in laureates]


def serialize_payload_laureates_object(laureate: Laureates) -> dict:
    return {
        'id': str(random.randint(0, 99999999)),
        'firstname': str(laureate.firstname),
        'surname': str(laureate.surname),
        'motivation': str(laureate.motivation),
        'share': str(laureate.share)
    }

In [5]:
from fastapi import APIRouter
from models.NobelPrizeModel import NobelPrize
from schemas.nobel_prize_schema import nobel_prizes_serializer, serialize_payload
from bson import ObjectId
from config.db import collection

nobel_prize_routes = APIRouter()


@nobel_prize_routes.get("/nobel-prize")
async def find_all_nobel_prize(category: str | None = None, year: str | None = None):
    query = {}
    if category is not None:
        query['category'] = category
    if year is not None:
        query['year'] = year
    data: list[NobelPrize] = nobel_prizes_serializer(collection.find(query))

    return {"status": "OK", "total": len(data), "data": data}


@nobel_prize_routes.get("/nobel-prize/{id}")
async def find_nobel_prize_by_id(_id: str):
    data: list[NobelPrize] = nobel_prizes_serializer(collection.find({'_id': ObjectId(_id)}))
    return {"status": "OK", "data": data}


@nobel_prize_routes.post("/nobel-prize")
async def save_nobel_prize(payload: NobelPrize):
    payload = serialize_payload(payload)
    response = collection.insert_one(dict(payload))
    nobel_prize = nobel_prizes_serializer(collection.find({"_id": response.inserted_id}))
    return {"status": "Ok", "acknowledged": response.acknowledged, "data": nobel_prize}

In [6]:
from fastapi import FastAPI

app = FastAPI()

app.include_router(nobel_prize_routes)

@app.get("/health")
async def root():
    return {"health": "OK"}